In [7]:
print("\033[1m   \033[0m")                                                         #create a blank row
print("\033[1m   Team Project - Bootstrap Sampling:\033[0m")                       #assign answer number
print("\033[1m   \033[0m")                                                         #create a blank row
import numpy as np
import pandas as pd
file_path = "C:/Users/Isaac/Desktop/Car-Dataset.xlsx"                              #Load data file
df = pd.read_excel(file_path, sheet_name="car_price_dataset")                      #Read data file
n_bootstrap = 1000                                                                 #Bootstrap samples
                                                                                   #Generate bootstrap samples to calce mean
bootstrap_means = [
    np.mean(np.random.choice(df["Price"], size=len(df), replace=True))
    for _ in range(n_bootstrap)
]
lower_bound, upper_bound = np.percentile(bootstrap_means, [2.5, 97.5])              #Confidence interval 95%
print(f"Estimated Mean Price (Bootstrap): {np.mean(bootstrap_means):.2f}")                      #Display results
print(f"95% Confidence Interval: ({lower_bound:.2f}, {upper_bound:.2f})")
                                                                                    #Interpret results 
print(f"""
The number of Bootstrap sample generated are 1000. 

Your bootstrap sampling results suggest that the estimated mean car price is {np.mean(bootstrap_means):.2f}, 95% Confidence 
Interval sure that the actual average price of all cars in the population is between the lower and upper bounds of 
the interval {lower_bound:.2f} and {upper_bound:.2f}.

Both results are quite close, and they suggest that the true mean car price is expected to fall within a narrow range around 
{np.mean(bootstrap_means):.2f}.

In summary, the bootstrap method gives an estimate of the mean price and shows how precise that estimate is based on the confidence interval.

""")


   
   Team Project - Bootstrap Sampling:
   
Estimated Mean Price (Bootstrap): 8851.17
95% Confidence Interval: (8788.79, 8912.78)

The number of Bootstrap sample generated are 1000. 

Your bootstrap sampling results suggest that the estimated mean car price is 8851.17, 95% Confidence 
Interval sure that the actual average price of all cars in the population is between the lower and upper bounds of 
the interval 8788.79 and 8912.78.

Both results are quite close, and they suggest that the true mean car price is expected to fall within a narrow range around 
8851.17.

In summary, the bootstrap method gives an estimate of the mean price and shows how precise that estimate is based on the confidence interval.




In [19]:
print("\033[1m   \033[0m")                                                         #create a blank row
print("\033[1m   Team Project - Bayesian Model:\033[0m")                           #assign answer number
print("\033[1m   \033[0m")                                                         #create a blank row
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import jax.numpy as jnp
import jax.random as random
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
from scipy import stats
import warnings
                                                                                  #Suppress warnings
warnings.filterwarnings("ignore", message="There are not enough devices to run parallel chains")
                                                                                  #Set available devices to 1 for execution
numpyro.set_host_device_count(1)
file_path = "C:/Users/Isaac/Desktop/Car-Dataset.xlsx"                             #Load data
df = pd.read_excel(file_path)                                                     #Read data
price_data = df["Price"].dropna().values                                          #Remove missing values
price_mean = np.mean(price_data)
price_std = np.std(price_data)
normalized_prices = (price_data - price_mean) / price_std
def bayesian_model():                                                             #Bayesian model
    mu = numpyro.sample("mu", dist.Normal(0, 1))                                  #For mean
    sigma = numpyro.sample("sigma", dist.HalfNormal(1))                           #For std deviation
    numpyro.sample("obs", dist.Normal(mu, sigma), obs=jnp.array(normalized_prices))
rng_key = random.PRNGKey(0)                                                       #Markov Chain Monte Carlo in Bayesian Inference
nuts_kernel = NUTS(bayesian_model)  
mcmc = MCMC(nuts_kernel, num_samples=2000, num_warmup=500, num_chains=2)          #Generate 2000 samples
mcmc.run(rng_key)
                                                                                  #Posterior samples
posterior_samples = mcmc.get_samples()
mu_post = posterior_samples["mu"] * price_std + price_mean                        #Convert to original scale
sigma_post = posterior_samples["sigma"] * price_std
mu_mean = np.mean(mu_post)                                                        #mean of posterior samples
sigma_mean = np.mean(sigma_post)                                                  #std from the posterior samples

#print("Raw Posterior mu mean samples (normalized):", posterior_samples["mu"][:500])  # First 10 samples for inspection
#print("Raw Posterior sigma standard deviation samples (normalized):", posterior_samples["sigma"][:500])  # First 10 samples for inspection
                                                                                  # Print results
print(f"Estimated Mean Price (Bayesian): ${mu_mean:.2f}")
print(f"Estimated Standard Deviation: ${sigma_mean:.2f}")


print(f"""
The average car price is estimated to be approximately ${mu_mean:.2f}.
The standard deviation of the car prices is estimated to be about $3,113.57, meaning that there is considerable 
variation in the prices (about 35% of the mean price).

The posterior mu values being close to zero after running 2000 samples in the normalized space confirm that the estimated mean price is close 
to the original mean price of the dataset, with only small deviations in the posterior samples.

The posterior sigma values being close to 1 after running 2000 samples in the normalized space confirm that the estimated standard deviation is 
quite consistent with the normalization process, and the standard deviation of the car prices is also close to the 
original spread of the data.

In summary, the Bayesian model estimates the mean price of cars to be around ${mu_mean:.2f} with a standard deviation of about ${sigma_mean:.2f}, 
suggesting moderate variability in the prices. The model's posterior estimates are quite stable and suggest that the average car
price is not far from the original dataset's mean price.

""")


   
   Team Project - Bayesian Model:
   


sample: 100%|██████████████████████████| 2500/2500 [00:00<00:00, 3054.83it/s, 3 steps of size 8.70e-01. acc. prob=0.89]

Estimated Mean Price (Bayesian): $8852.95
Estimated Standard Deviation: $3113.57

The average car price is estimated to be approximately $8852.95.
The standard deviation of the car prices is estimated to be about $3,113.57, meaning that there is considerable 
variation in the prices (about 35% of the mean price).

The posterior mu values being close to zero after running 2000 samples in the normalized space confirm that the estimated mean price is close 
to the original mean price of the dataset, with only small deviations in the posterior samples.

The posterior sigma values being close to 1 after running 2000 samples in the normalized space confirm that the estimated standard deviation is 
quite consistent with the normalization process, and the standard deviation of the car prices is also close to the 
original spread of the data.

In summary, the Bayesian model estimates the mean price of cars to be around $8852.95 with a standard deviation of about $3113.57, 
suggesting moderate v